![openclassrooms](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/courses/6204541/1+HnqdJ-5ofxiPP9HIxdNdpw.jpeg)

# Agrégez des données avec Pandas

Suite aux résultats que vous avez fourni précédemment, le responsable du service prêt revient vous voir avec une remarque : vous l’avez probablement remarqué, mais certains clients ont contracté plusieurs prêts au sein de notre établissement. Cela fausse donc potentiellement les calculs réalisés précédemment.

Il formule donc des demandes additionnelles, auxquelles vous allez devoir répondre en appliquant diverses agrégations.

In [1]:
import numpy as np
import pandas as pd

In [7]:
prets = pd.read_csv('https://raw.githubusercontent.com/OpenClassrooms-Student-Center/fr-4452741-decouvrez-les-librairies-python-pour-la-data-science/main/data/prets.csv')

prets['taux_endettement'] = round(prets['remboursement'] * 100 / prets['revenu'], 2)

prets.rename(columns={'taux':'taux_interet'}, inplace=True)

prets['cout_total'] = prets['remboursement'] * prets['duree']

prets['benefices'] = round((prets['cout_total'] * prets['taux_interet']/100)/(24), 2)

prets['risque'] = 'Non'
prets.loc[prets['taux_endettement'] > 35, 'risque'] = 'Oui'

prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices,risque
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99,Non
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69,Non
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97,Non
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71,Non
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57,Non


Pourriez vous créez un dataframe profil client pour pallier ce problème ?

Par profil client, nous entendons un dataframe où il y n'y a qu'une seule ligne par client, avec le résumé de ses informations (somme remboursement, du taux d'endettement, du cout_total et des bénéfices réalisés)

In [18]:
profil_client = prets.groupby('identifiant').agg(
    remboursement_total=('remboursement', 'sum'),
    revenu=('revenu', 'first'),
    ville=('ville', 'first'),
    cout_total_global=('cout_total', 'sum'),
    benefice_total=('benefices', 'sum')
).reset_index()

profil_client['taux_endettement'] = round(
    (profil_client['remboursement_total'] * 100) / profil_client['revenu'], 2
)

profil_client = profil_client[['identifiant', 'ville', 'revenu', 'remboursement_total',
                               'taux_endettement', 'cout_total_global', 'benefice_total']]

profil_client.head()

,identifiant,ville,revenu,remboursement_total,taux_endettement,cout_total_global,benefice_total
0,0,TOULOUSE,3669.0,1130.05,30.80,271212.0,131.99
1,1,PARIS,5310.0,1487.85,28.02,389715.0,206.66
2,2,MARSEILLE,1873.0,552.54,29.50,132609.6,53.71
3,3,MARSEILLE,1684.0,586.03,34.80,105485.4,44.57
4,4,MARSEILLE,1476.0,423.61,28.70,101666.4,51.21


Recalculez le nombre exact de personnes en situation bancaire risquée à partir du taux d'endettemment :

In [19]:
clients_risque = profil_client[profil_client['taux_endettement'] > 35]

print(f"Nombre de clients en situation de risque bancaire : {len(clients_risque)}")


Nombre de clients en situation de risque bancaire : 62


Calculez le bénéfice dégagé par chacune de nos agences, par types de prêts. Vous présenterez vos résultats sous la forme d'un tableau simple (via un `group_by`) :

In [20]:
benef_par_agence_type = (prets.groupby(['ville', 'type'])['benefices']
                        .sum()
                        .unstack(fill_value=0)
                        .round(2))

benef_par_agence_type

type,automobile,immobilier
ville,,
BORDEAUX,0.00,1681.32
LYON,47.39,2967.41
MARSEILLE,61.36,4334.55
NICE,31.15,2651.60
PARIS,99.50,9916.85
TOULOUSE,61.22,2076.85


Pour aller plus loin, on souhaite avoir un tableau à double entrée (via un `pivot_table`) présentant cette fois ci les bénéfices moyen réalisés par chaque agence, pour chaque type de prêt :

In [21]:
pivot_benefice_moyen = pd.pivot_table(prets,
                                      values='benefices',
                                      index='ville',
                                      columns='type',
                                      aggfunc='mean',
                                      fill_value=0).round(2)

pivot_benefice_moyen

type,automobile,immobilier
ville,,
BORDEAUX,0.00,105.08
LYON,15.80,118.70
MARSEILLE,20.45,60.20
NICE,15.58,139.56
PARIS,19.90,137.73
TOULOUSE,20.41,86.54


Quelle ville semble la plus intéressante où développer les prêts immobiliers ?

Félicitation pour être arrivé au bout ! Voici [la correction correspondante](https://colab.research.google.com/github/OpenClassrooms-Student-Center/fr-4452741-decouvrez-les-librairies-python-pour-la-data-science/blob/main/notebooks/P2/P2C4%20-%20Agr%C3%A9gez%20des%20donn%C3%A9es%20avec%20Pandas%20-%20correction.ipynb).

In [16]:
top_ville_immo = pivot_benefice_moyen['immobilier'].sort_values(ascending=False)

print("Bénéfice moyen par prêt immobilier par ville :")
print(top_ville_immo)

print(f"\nLa ville la plus rentable pour développer les prêts immobiliers est : {top_ville_immo.index[0].upper()}")
print(f"avec un bénéfice moyen de {top_ville_immo.iloc[0]:.2f} € par prêt")

Bénéfice moyen par prêt immobilier par ville :
ville
NICE         139.56
PARIS        137.73
LYON         118.70
BORDEAUX     105.08
TOULOUSE      86.54
MARSEILLE     60.20
Name: immobilier, dtype: float64

La ville la plus rentable pour développer les prêts immobiliers est : NICE
avec un bénéfice moyen de 139.56 € par prêt
